# Nova Grounding (Web Search) with LangChain

This notebook demonstrates how to use Amazon Nova 2.0's web grounding system tool with LangChain's ChatBedrockConverse.

System tools are different from standard tools - the model can utilize these tools throughout its reasoning process and actually invoke the tool itself. This is in contrast with standard tool workflows where the model returns a tool call and the developer is responsible for invocation.

The Nova Grounding system tool enables the Nova 2.0 model to access the web and search for live information.

## Setup

Install the required packages:

In [ ]:
%pip install -qU langchain-aws

## IAM Permissions

**Important:** To invoke system tools, your IAM role must have the `bedrock:InvokeTool` permission in addition to `bedrock:InvokeModel`.

## Basic Web Search Example

In [ ]:
from botocore.config import Config
from langchain_aws import ChatBedrockConverse
from langchain_aws.tools import NovaGroundingTool

# Configure extended timeouts
config = Config(
    connect_timeout=3600,  # 60 minutes
    read_timeout=3600,     # 60 minutes
    retries={'max_attempts': 1}
)

# Initialize model
model = ChatBedrockConverse(
    model="amazon.nova-2-lite-v1:0",
    region_name="us-east-1",
    temperature=0.7,
    max_tokens=10000,
    config=config,
    additional_model_request_fields={
        "reasoningConfig": {
            "type": "enabled",
            "maxReasoningEffort": "low"
        }
    }
)

# Bind the web grounding tool
model_with_search = model.bind_tools([NovaGroundingTool()])

In [ ]:
# Ask a question that requires current information
response = model_with_search.invoke("Who won the oscar for best actress?")
print(response.content)

## Alternative: Using String Name

You can also use the string name directly:

In [ ]:
# Using string name instead of tool class
model_with_search = model.bind_tools(["nova_grounding"])

response = model_with_search.invoke("What are the latest developments in quantum computing?")
print(response.content)

## Understanding the Response

Let's examine the different parts of the response:

In [ ]:
# The response contains multiple content blocks
for i, block in enumerate(response.content):
    if isinstance(block, dict):
        print(f"\n=== Block {i} ===")
        if 'type' in block:
            print(f"Type: {block['type']}")
            
            if block['type'] == 'reasoning_content':
                print("Reasoning:", block['reasoning_content']['text'][:200], "...")
            elif block['type'] == 'tool_use':
                print("Tool:", block.get('name'))
                print("Tool Use Type:", block.get('type'))
            elif block['type'] == 'tool_result':
                print("Tool Result Status:", block.get('status'))
                # Note: content is redacted in the final response for privacy
            elif block['type'] == 'text':
                print("Final Answer:", block['text'])

## Streaming with Web Search

You can also stream responses to see the reasoning, tool use, and final answer as they're generated:

In [ ]:
print("Streaming response:\n")

current_type = None
for chunk in model_with_search.stream("Who is going to the 2025 MLB world series?"):
    if chunk.content:
        for block in chunk.content:
            if isinstance(block, dict) and 'type' in block:
                # Print header when type changes
                if block['type'] != current_type:
                    current_type = block['type']
                    print(f"\n\n=== {current_type.upper()} ===")
                
                # Stream the content
                if block['type'] == 'reasoning_content':
                    print(block['reasoning_content'].get('text', ''), end='', flush=True)
                elif block['type'] == 'text':
                    print(block.get('text', ''), end='', flush=True)
                elif block['type'] == 'tool_use':
                    if 'name' in block:
                        print(f"Tool: {block['name']}")

## Combining with Other Features

You can combine web search with other Nova features like structured output:

In [ ]:
from pydantic import BaseModel, Field
from typing import List

class NewsUpdate(BaseModel):
    """Current news information"""
    topic: str = Field(description="The main topic")
    key_points: List[str] = Field(description="Key points from the news")
    date_context: str = Field(description="When this information is relevant")

# Create a model with both web search and structured output
structured_model = model_with_search.with_structured_output(NewsUpdate)

result = structured_model.invoke("What are the latest AI breakthroughs this month?")
print(f"Topic: {result.topic}")
print(f"\nKey Points:")
for point in result.key_points:
    print(f"  - {point}")
print(f"\nDate Context: {result.date_context}")

## Multi-turn Conversations

Web search works seamlessly in multi-turn conversations:

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

# First turn
messages = [HumanMessage(content="What's the weather like in San Francisco today?")]
response1 = model_with_search.invoke(messages)
print("Assistant:", response1.content)

# Second turn - follow up question
messages.append(response1)
messages.append(HumanMessage(content="What about tomorrow?"))
response2 = model_with_search.invoke(messages)
print("\nAssistant:", response2.content)